In [1]:
import pycalibration
pycalibration.install()

In [2]:
import numpy as np
from pycalibration import ca
import time

In [7]:
def SKCE(kernel, predictions, outcomes):

    predictions = np.array(predictions)
    outcomes = np.array(outcomes)
    n = len(outcomes)

    # Initialize SKCE
    skce_matrix = np.zeros((n,n))

    # For each pair of data points
    for i in range(n):
        for j in range(i+1,n):
            # Compute the difference between unit vectors outcomes and predictions
            diff_Yi_g_Xi = np.array([outcomes[i] - predictions[i]])
            diff_Yj_g_Xj = np.array([outcomes[j] - predictions[j]])

            # Compute the kernel matrix for the pair
            k_g_Xi_g_Xj = kernel(predictions, predictions)

            matrix_kernel = k_g_Xi_g_Xj[i,j] * np.eye(len(predictions[i]))

            skce_matrix[i,j] = ((diff_Yi_g_Xi @ matrix_kernel) @ diff_Yj_g_Xj.T)[0,0]

    skce = np.sum(skce_matrix)

    
    # Divide the SKCE by n over 2
    prefactor = (n* (n-1)) // 2
    skce /= prefactor

    return skce


def transform_outcomes(predictions, outcomes):
    #creates vector for outcomes to match predictions shape
    outcomes_minus_one = [x - 1 for x in outcomes]
    transformed_outcomes = np.zeros_like(predictions)
    transformed_outcomes[np.arange(len(predictions)), outcomes_minus_one] = 1
    return transformed_outcomes

# Use the rbf_kernel function from sklearn.metrics.pairwise
def rbf_pw(X, Y=None, length_scale=1.0):
    gamma = 1.0 / (2 * length_scale ** 2)
    return RBF_PW(X, Y, gamma=gamma)

In [ ]:
def SKCE_torch(kernel, predictions, outcomes):
       predictions = torch.tensor(predictions, dtype=torch.float64)
       outcomes = torch.tensor(outcomes, dtype=torch.float64)
       n = len(outcomes)
       diff_Y_g_X = outcomes - predictions

       # kernel matrix
       k_g_Xi_g_Xj = kernel(predictions, predictions)

       # Generate all pairwise combinations of indices
       i_indices, j_indices = torch.triu_indices(n, n, 1)

       # Create an identity matrix for matrix_kernel
       identity_matrix = torch.eye(predictions.shape[1], dtype=torch.float32)

       # Select the pairwise differences
       diff_Yi_g_Xi = diff_Y_g_X[i_indices]
       diff_Yj_g_Xj = diff_Y_g_X[j_indices]

       # Compute the matrix kernel for all pairs
       # k_g_Xi_g_Xj[i_indices, j_indices] Selects Upper Triangle Pairs
       matrix_kernels = k_g_Xi_g_Xj[i_indices,j_indices].unsqueeze(-1).unsqueeze(-1) * identity_matrix
       print(matrix_kernels.shape)
       # Compute the SKCE matrix by first Yi_Xi @ Upper Triangle and then @ Yj_Xj
       skce_values = (diff_Yi_g_Xi.unsqueeze(1) @ matrix_kernels @ diff_Yj_g_Xj.unsqueeze(-1)).reshape(-1)

       # Sum the SKCE values
       skce = torch.sum(skce_values)

       # Normalize the SKCE value
       prefactor = (n * (n - 1)) // 2
       skce /= prefactor

       return skce

In [8]:
from sklearn.gaussian_process.kernels import RBF as RBF_GP
from sklearn.metrics.pairwise import rbf_kernel as RBF_PW

# Instantiate the RBF kernel from sklearn.gaussian_process.kernels
rbf_gp = RBF_GP(length_scale=1)

# Use the rbf_kernel function from sklearn.metrics.pairwise
def rbf_pw(X, Y=None, length_scale=1.0):
    gamma = 1.0 / (2 * length_scale ** 2)
    return RBF_PW(X, Y, gamma=gamma)

In [9]:
rng = np.random.default_rng(1234)
import random
rng = np.random.default_rng(1234)
predictions = [rng.dirichlet((3, 2, 5)) for _ in range(40)]
outcomes = rng.integers(low=1, high=4, size=40)
predictions = np.array([[0.1,0.9],[0.9,0.1]])
outcomes = np.array([2,1])
predictions = np.array([[0.5,0.5],[0.5,0.5]])
outcomes = np.array([2,2])



#2 Datenpukte 3 Klassen
predictions = np.array([[1/3,1/3,1/3],[1/3,1/3,1/3]])
outcomes = np.array([3,1])

predictions = np.array([[0.5,0.5],[0.5,0.5],[0.5,0.5]])
outcomes = np.array([2,1,1])
#outcomes = predictions.copy()
predictions = np.array([[0.25,0.75],[0.5,0.5],[0.75,0.25]])
outcomes = np.array([2,2,1])
predictions = np.array([[1/3,1/3,1/3],[1/3,1/3,1/3]])
outcomes = np.array([3,1])


#print(transform_outcomes(predictions, outcomes))

# Now you can use these in your calculate_SKCE function
SKCE(rbf_pw, predictions, transform_outcomes(predictions, outcomes))
#print(SKCE(rbf_pw, predictions, outcomes))


-0.33333333333333337

In [10]:
rbf_pw(predictions, predictions)

array([[1., 1.],
       [1., 1.]])

In [11]:
#skce_julia = ca.SKCE(ca.RBFKernel(), unbiased=True)
skce_julia = ca.SKCE(ca.tensor(ca.RBFKernel(), ca.WhiteKernel()), unbiased = True)
skce_julia(ca.RowVecs(predictions), outcomes)

-0.3333333333333333

In [38]:
#skce_julia = ca.SKCE(ca.RBFKernel(), unbiased=True)
#skce_julia(predictions, outcomes)

[0 0 0 0 1 1 1 2 2 3]


In [24]:
# Measure the time taken by function1
start_time = time.time()
SKCE(rbf_gp, predictions, outcomes)
end_time = time.time()
print(f"SKCE Python took {end_time - start_time} seconds to execute.")

# Measure the time taken by function2
start_time = time.time()
skce_julia(ca.RowVecs(predictions), outcomes)
end_time = time.time()
print(f"SKCE Julia took {end_time - start_time} seconds to execute.")

0
0
1
0
2
1
2


TypeError: unsupported operand type(s) for +: 'NoneType' and 'NoneType'

In [93]:
from julia import Main
import numpy as np

def test_rbf_kernel(predictions):
    Main.eval("""
    using KernelFunctions
    rbf = SqExponentialKernel()
    """)

    kernel_matrix = np.zeros((len(predictions), len(predictions)))

    for i in range(len(predictions)):
        for j in range(i+1,len(predictions)):
            kernel_matrix[i][j] = Main.eval(f"rbf({list(predictions[i])}, {list(predictions[j])})")

    return kernel_matrix

# Test the function with two predictions
#predictions = np.array([[0.1,0.9],[0.9,0.1],[0.5,0.5]])
print(test_rbf_kernel(predictions))


-0.03245003262797521

In [94]:
rbf_pw(predictions, predictions)

-0.03245003262797521

In [13]:
Main.eval(f"rbf({list(predictions[0])}, {list(predictions[1])})")

0.9394130628134758